In [1]:
%load_ext autoreload
%autoreload 2

import os
from tqdm import tqdm

from processingmm import batch_processing
from freezingeffect import selection_of_ROIs, automatic_ROI_propagation, collect_data_propagated
from freezingeffect.helpers import load_parameters_ROIs

## 1. Process the Mueller Matrices of the folders

In [2]:
# set the parameters to run the script
data_folder_path = r'F:\ImpactHistology\Freezing_dev'
directories = [data_folder_path]
calib_directory = r'F:\polarimetry\calibration'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'PD')

0it [00:00, ?it/s]


[]

## 2. Specify the folders that will be processed and clean them (remove precedent processing results)

In [3]:
path_folder_temp, path_align_folder = selection_of_ROIs.create_alignment_folder()
base_dirs = selection_of_ROIs.get_the_base_dirs(data_folder_path)
selection_of_ROIs.clean_the_alignment_folders(path_align_folder)
selection_of_ROIs.create_the_masks(data_folder_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:16<00:00,  1.38s/it]


## 3. Loop over all the folders that will be processed

#### 3.1. loop over all the folders that needs to be processed

In [4]:
from datetime import datetime
now = datetime.now()

In [5]:
now

datetime.datetime(2023, 6, 30, 17, 23, 16, 439998)

In [6]:
now.strftime("%d/%m/%Y %H:%M:%S")

'30/06/2023 17:23:16'

In [7]:
# tissue_types = ['WM', 'GM']
tissue_types = ['WM', 'GM']
for path_folder in tqdm(base_dirs):
    propagation_lists = {}
    
    for tissue_type in tissue_types:
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_lists[tissue_type] = selection_of_ROIs.square_selection(path_folder_temp, path_folder, WM, 
                                                                            tissue_type, path_align_folder)
       
    # 3. actually do the alignment
    propagation_lists = automatic_ROI_propagation.generate_combined_mask(propagation_lists, path_align_folder)
    automatic_ROI_propagation.do_alignment(path_align_folder)
    output_folders = automatic_ROI_propagation.move_computed_folders(path_align_folder)
    
    # 4. and collect the data after propagation
    for tissue_type in tissue_types:
        WM = tissue_type == 'WM'
        collect_data_propagated.collect_data_propagated(WM, path_align_folder, propagation_lists[tissue_type], output_folders)

 17%|█████████████▊                                                                     | 1/6 [04:15<21:17, 255.57s/it]C:\Users\romai\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\romai\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\romai\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\romai\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\romai\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:257: RuntimeWarnin

## 4. Compare the parameters

#### 4.1. Get the folders for the measurements of interest

In [21]:
from freezingeffect import parameter_comparaison

In [22]:
# get the parameters for the comparaison
path_folder = data_folder_path
wavelength, match_sequence, measurement, max_, max_nb_of_squares = parameter_comparaison.get_parameters()
max_nb_of_squares = max_nb_of_squares * len(base_dirs)


# and the folders containing the data
data_folders, base_folders, to_be_removed = parameter_comparaison.get_data_folders(path_folder, wavelength, match_sequence)

data_folders[0:5] + data_folders[-5:]

['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_1',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_10',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_11',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_12',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_13',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_5',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_6',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_7',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_8',
 'F:\\

#### 4.2. Get the data for the measurement

In [23]:
data_types, data_fixation = parameter_comparaison.get_data(data_folders)

data_fixation['GM']['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2']

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,117.234690,19.418124,114.636872,117.234690,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.796827,0.022342,0.792647,0.796655,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.032110,0.010525,0.025253,0.031237,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,14.191998,5.625820,14.581006,14.196521,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,21.478132,35.639395,57.318436,21.478132,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.893959,0.028525,0.897794,0.896853,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.056426,0.014064,0.050505,0.055628,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,15.471984,9.103367,10.558659,13.379551,None


In [24]:
# remove the ROI moving from white matter to grey matter and vice versa
for data_type, dfed in data_fixation.items():
    for folder, df in dfed.items():
        to_remove_dfed = to_be_removed[folder.split('\\polarimetry')[0]]
        for val in to_remove_dfed:
            for fol in val:
                if fol[0] == folder.split('\\')[-1].split('_align')[0]:
                    df = df.apply(parameter_comparaison.find_match_seq, match_seq = fol[1], axis = 1)
                    data_fixation[data_type][folder] = df

In [25]:
data_fixation['GM']['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2']

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,117.234690,19.418124,114.636872,117.234690,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.796827,0.022342,0.792647,0.796655,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.032110,0.010525,0.025253,0.031237,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,14.191998,5.625820,14.581006,14.196521,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,NaN,NaN,NaN,NaN,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,NaN,NaN,NaN,NaN,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,NaN,NaN,NaN,NaN,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,NaN,NaN,NaN,NaN,None


In [26]:
#### Set up the parameters to be studied
parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth']
metrics = ['median', 'mean', 'max']
times = ['T0', 'T1']
    
# create the directories that will be used to store the output of the analysis
folder = parameter_comparaison.create_output_directories(measurement, data_types, parameters = parameters)

In [27]:
### Get the data and save it into xlsx file
to_combine = parameter_comparaison.get_and_organize_data(data_fixation, metrics, data_types, times, folder, max_,
                          parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth'])

to_combine[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,9.790383,16.828685,15.627832,11.382821,17.780420,11.341142,27.820179,29.088053,12.099045,10.337447,...,16.108336,17.201869,13.695809,13.922080,16.309019,19.529999,14.420564,14.449318,19.832664,20.447373
1,16.044143,NaN,17.148179,20.908097,23.862015,15.347278,20.885637,22.836430,12.196142,14.098137,...,19.080227,18.062118,11.565799,13.997067,14.408819,11.979884,19.299942,14.646874,13.404878,18.443470


In [28]:
to_combine_cv = {}
for idx, val in to_combine.items():
    if idx[0] == 'azimuth':
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times), azimuth = True)
    else:
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times))
        
to_combine_cv[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.638766,NaN,1.097285,1.836812,1.342039,1.353239,0.750737,0.785079,1.008025,1.363793,...,1.184494,1.050009,0.844477,1.005386,0.883488,0.613409,1.338362,1.013672,0.675899,0.901997


#### 4.3. Statistical analysis

In [29]:
#### Get the data to be used for the comparisons
# recombine the data in a different format
data_all_recombined = parameter_comparaison.recombine_data_tests(to_combine, times)
data_all_recombined_cv = parameter_comparaison.recombine_data_tests(to_combine_cv, times)

In [30]:
#### Comparaison of the means and the means of the fold change
# perform the statistical test for both fold change and mean values
paired_test = parameter_comparaison.t_test(data_all_recombined, times, 'median')# times[1:]])
paired_test_cv = parameter_comparaison.t_test(data_all_recombined_cv, times, 'median')# times[1:]])
df_grouped = parameter_comparaison.create_test_df(paired_test, parameter = 'median')
df_grouped_cv = parameter_comparaison.create_test_df(paired_test_cv, parameter = 'median')
df_grouped_cv

Z       p_value    n_1    n_2  before/GM  \
parameter      data_type time                                                   
azimuth        GM        T0    19500.0  1.953051e-55  130.0  150.0  30.370125   
                         T1     8450.0  1.000000e+00  130.0  130.0  30.370125   
               WM        T0    21150.0  1.077133e-56  141.0  150.0  28.172206   
                         T1     9940.5  1.000000e+00  141.0  141.0  28.172206   
depolarization GM        T0    10200.0  4.696190e-01  130.0  150.0   1.008334   
                         T1     8450.0  1.000000e+00  130.0  130.0   1.008334   
               WM        T0    14550.0  2.467091e-09  141.0  150.0   1.003135   
                         T1     9940.5  1.000000e+00  141.0  141.0   1.003135   
retardance     GM        T0     5700.0  7.301619e-11  130.0  150.0   0.840229   
                         T1     8450.0  1.000000e+00  130.0  130.0   0.840229   
               WM        T0    10950.0  5.741663e-01  141.0  150.0   1.004894   
                         T1     9940.5  1.000000e+00  141.0  141.0   1.004894   

                               before/GM std   after/WM  after/WM std  
parameter      data_type time                                          
azimuth        GM        T0        26.357007   0.000000      0.000000  
                         T1        26.357007  30.370125     26.357007  
               WM        T0        23.191157   0.000000      0.000000  
                         T1        23.191157  28.172206     23.191157  
depolarization GM        T0         0.041938   1.000000      0.000000  
                         T1         0.041938   1.008334      0.041938  
               WM        T0         0.012575   1.000000      0.000000  
                         T1         0.012575   1.003135      0.012575  
retardance     GM        T0         0.372067   1.000000      0.000000  
                         T1         0.372067   0.840229      0.372067  
               WM        T0         0.297596   1.000000      0.000000  
                         T1         0.297596   1.004894      0.297596

In [31]:
# finally, save the results
path_statistical_test = os.path.join('results', measurement)
df_grouped.to_excel(os.path.join(path_statistical_test, 'statistical_test.xlsx'))
df_grouped_cv.to_excel(os.path.join(path_statistical_test, 'statistical_test_cv.xlsx'))

#### 4.4. Combine the results in a single excel file

In [32]:
# this cell allows to combine the data obtained for both grey and white matter in a table that can then be used
# to plot the results using Graphpad Prism
retardance = {}
depolarization = {}
for key, param in to_combine.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization[key[1]] = param
        
retardance_cv = {}
depolarization_cv = {}
azimuth_cv = {}
for key, param in to_combine_cv.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance_cv[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization_cv[key[1]] = param
    else:
        if key[2] == 'mean':
            azimuth_cv[key[1]] = param
        
retardance = parameter_comparaison.create_df_prism(retardance, max_nb_of_squares)
depolarization = parameter_comparaison.create_df_prism(depolarization, max_nb_of_squares)
    
retardance_cv = parameter_comparaison.create_df_prism(retardance_cv, max_nb_of_squares)
depolarization_cv = parameter_comparaison.create_df_prism(depolarization_cv, max_nb_of_squares)
azimuth_cv = parameter_comparaison.create_df_prism(azimuth_cv, max_nb_of_squares)

retardance_cv

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.369633,0.197303,1.286178,0.677256,NaN,NaN,0.666799,1.126001,1.302472,0.353366,...,1.184494,1.050009,0.844477,1.005386,0.883488,0.613409,1.338362,1.013672,0.675899,0.901997


In [33]:
# finally save all the tables
path_data_prism = os.path.join('results', measurement)
retardance.to_excel(os.path.join(path_data_prism, "retardance_prism.xlsx"))
depolarization.to_excel(os.path.join(path_data_prism, "depolarization_prism.xlsx"))
retardance_cv.to_excel(os.path.join(path_data_prism, "retardance_prism_cv.xlsx"))
depolarization_cv.to_excel(os.path.join(path_data_prism, "depolarization_prism_cv.xlsx"))
azimuth_cv.to_excel(os.path.join(path_data_prism, "azimuth_prism_cv.xlsx"))